In [15]:
import pandas as pd
import sqlite3

# 파일 경로
places_path = "C:/Users/junlo/Desktop/졸업 프로젝트/food_카페음식점분류 (1).csv"
tour_path = "C:/Users/junlo/Desktop/졸업 프로젝트/tour.csv"

# 인코딩 확인 (보통 cp949 또는 euc-kr)
df_places = pd.read_csv(places_path, encoding="UTF-8")
df_tour = pd.read_csv(tour_path, encoding="euc-kr")

# DB 연결
conn = sqlite3.connect("C:/jeju_project/jejudb.db")
cursor = conn.cursor()

# 기존 테이블 제거
cursor.execute("DROP TABLE IF EXISTS places")
cursor.execute("DROP TABLE IF EXISTS tour")

# ✅ places 테이블 생성 (Primary Key 포함)
cursor.execute("""
CREATE TABLE places (
    id INTEGER PRIMARY KEY,
    name TEXT,
    address TEXT,
    lat REAL,
    lng REAL,
    category TEXT,
    business_hours TEXT,
    closed_day TEXT,
    main_menu TEXT,
    menu TEXT,
    parking TEXT
)
""")

# ✅ tour 테이블 생성 (Primary Key 포함)
cursor.execute("""
CREATE TABLE tour (
    id INTEGER PRIMARY KEY,
    name TEXT,
    theme TEXT,
    theme_category TEXT,
    address TEXT,
    address_detail TEXT,
    lat REAL,
    lng REAL
)
""")
columns_to_keep = [
    "ID", "명칭", "주소", "위도", "경도", "카페/음식점",
    "영업시간", "쉬는날", "대표메뉴", "취급메뉴", "주차 시설"
]

df_places_filtered = df_places[columns_to_keep].copy()

# 열 이름을 영어로 변환해서 DB 테이블 구조 일관성 있게 변경
df_places_filtered.columns = [
    "id", "name", "address", "lat", "lng", "category",
    "business_hours", "closed_day", "main_menu", "menu", "parking"
]
df_tour['id'] = pd.to_numeric(df_tour['id'], errors='coerce').astype("Int64")
columns = ['id', 'name', 'theme', 'theme_category', 'address', 'address_detail', 'lat', 'lng']
df_tour_filtered = df_tour[columns]
# CSV → SQLite 삽입
df_places_filtered.to_sql("places", conn, if_exists="append", index=False)
df_tour_filtered.to_sql("tour", conn, if_exists="append", index=False)

# 마무리
conn.commit()
conn.close()


In [18]:
import sqlite3

# DB 연결
conn = sqlite3.connect("C:/jeju_project/jejudb.db")
cursor = conn.cursor()

# ✅ 추천 결과 테이블 생성
cursor.execute("""
CREATE TABLE IF NOT EXISTS recommendations (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id TEXT,
    selected_theme TEXT,
    place_ids TEXT,        -- 예: "2,4,7"
    total_distance REAL,
    timestamp TEXT DEFAULT (datetime('now', 'localtime'))
)
""")

# ✅ 리뷰 테이블 생성
cursor.execute("""
CREATE TABLE IF NOT EXISTS reviews (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id TEXT,
    place_id INTEGER,
    rating INTEGER,
    comment TEXT,
    timestamp TEXT DEFAULT (datetime('now', 'localtime'))
)
""")

# 저장 및 종료
conn.commit()
conn.close()


In [17]:
conn = sqlite3.connect("C:/jeju_project/jejudb.db")
query = "SELECT * FROM tour"
df=pd.read_sql(query, conn)
print(df.head())
# 대략 위도 33.2~33.4, 경도 126.3~126.5 구간

   id        name theme theme_category                    address  \
0   1   동산물감귤체험농장  체험마을          액티비티형  제주특별자치도 서귀포시 성산읍 수산리 1948   
1   2     제주돌마을공원    공원         자연/경치형  제주특별자치도 제주시 한림읍 금능리 368-2   
2   3      제주마방목지  체험마을          액티비티형    제주특별자치도 제주시 용강동  산14-34   
3   4  우도등대공원우도등대    공원         자연/경치형    제주특별자치도 제주시 우도면 연평리 337   
4   5      제주드림랜드  체험마을          액티비티형   제주특별자치도 제주시 애월읍 봉성리 4554   

  address_detail        lat         lng  
0       서성일로 809  33.436260  126.867227  
1       금능남로 421  33.353791  126.253694  
2      516로 2480  33.427697  126.603715  
3       우도봉길 105  33.492579  126.965790  
4       평화로 1529  33.363018  126.364109  


In [ ]:
conn = sqlite3.connect("C:/jeju_project/jejudb.db")
query = "SELECT * FROM places WHERE lat BETWEEN 33.2 AND 33.4 AND lng BETWEEN 126.3 AND 126.5"
df=pd.read_sql(query, conn)
print(df.head())
# 대략 위도 33.2~33.4, 경도 126.3~126.5 구간

        id           name                           address        lat  \
0  1018723      하영 흑돼지구이집            제주특별자치도 서귀포시 천제연로 101   33.255075   
1  2384971           지오아라  제주특별자치도 서귀포시 안덕면 사계로114번길 54-86   33.236351   
2  2561772  스카이 온 파이브 다이닝   제주특별자치도 서귀포시 안덕면 신화역사로304번길 38   33.304951   
3  2561780         랜딩 다이닝   제주특별자치도 서귀포시 안덕면 신화역사로304번길 38   33.304951   
4  2561782          랜딩 델리   제주특별자치도 서귀포시 안덕면 신화역사로304번길 38   33.304951   

          lng category                                  business_hours  \
0  126.416596      음식점                      10:00~22:00 (마지막 주문 21:00)   
1  126.307828       카페                                     11:00~23:00   
2  126.316404      음식점  06:30~21:30 (브레이크타임 10:30~12:00 / 14:30~18:00)   
3  126.316404      음식점                07:00~21:30 (브레이크타임 11:00~18:00)   
4  126.316404       카페                                     08:00~20:00   

  closed_day      main_menu                         menu parking  
0       연중무휴       흑돼지 모둠한판    고사리강된장 / 전복버